In [6]:
import pandas as pd
import os
from plotnine import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, accuracy_score
import warnings
warnings.filterwarnings("ignore")



### Ingrese aquí el nombre de los integrantes del grupo
- Nombre integrante 1
- 
- 
- 

# Descripción del problema y la data
La data acompañada corresponde a clientes de una cadena de ropa, y su comportamiento con respecto a campañas de marketing que se le envian.  La `data` contiene 16305 clientes (filas), y 44 *features* (columnas), indicando su número de visitas a la tienda, las ventas, porcentaje de compras en los distintos items, montos gastados en distintos locales de la tienda y en distintos rangos de meses, etc. A estos clientes se les envió una campaña de marketing directa, ante la cual ellos respondieron (`target` = 1) o no lo hicieron (`target`=0).

Hacer una campaña de marketing directo a estos clientes puede tener un impacto importante en las ventas. Se estima que cada cliente que responde a estas campañas genera un profit de $\$28.40$ para la empresa. Sin embargo, hacer estas campañas tiene un costo estimado de $\$2.00$ por cliente.  Es por esto, que la empresa desea identificar correctamente aquellos clientes que son mas proclives a responder a este tipo de campaña de marketing.

El objetivo de este trabajo entonces es construir un **modelo de clasificación**, que permita predecir si un cliente va a responder o no a una campaña de marketing.



Para esto:
- Decida si utilizará todos los features o no. Puede eliminar features, o agregar nuevos si lo desea.  Explique en este documento lo que hizo al respecto.
- Transforme la data si es necesario. Note que los features tienen distintas escalas (por ejemplo, algunos son binarios (0,1), otros son porcentajes [0-1], otros son días [0-365], y otros montos gastados [0-22511].  Explicite **claramente** la o las transformaciones necesarias a la data. Su modelo será evaluado con nueva data, por lo que es necesario tener claro las transformaciones necesarias para poder aplicarlo. 
- Escoga una (o más de una) métrica que utilizará para poder evaluar el performance de sus modelos. Indique cuál será la métrica prioritaria para su evaluación, y el por qué de esta decisión.
- Escoga entre al menos 3 modelos de clasificación estudiados en el curso. Para cada uno de ellos, realice una optimización de hiper-parámetros, para escoger los mejores parámetros para cada uno de ellos.  **Atención**: al menos unos de estos tres modelos debe poder ser interpretable.
- Compare estos tres (o mas) modelos escogidos (con los mejores parámetros que optimizó) entre ellos, usando cross-validation.  Decida cuál es el mejor modelo para usted. **Agregue este modelo al final de este documento** (ver ejemplo).
- Utilice el mejor modelo obtenido pero que tenga capacidad de interpretación de los resultados, y analice este mismo. ¿Cuáles son las características o features más importantes para que un cliente responda positivamente a una campaña de márketing directo?

In [4]:

def one_hot_encoder(df):
  """
  DOCSTRING
  """
  chunks = []
  chunks.append(df)
  bool_cols = df.select_dtypes(include= [bool]).columns
  
  for col in bool_cols:
    chunk = pd.get_dummies(data[col], prefix=col)
    chunks.append(chunk)
  
  df = pd.concat(chunks, axis= 1)
  df.drop(bool_cols, axis= 1, inplace= True)
  

  return df


def fitter(model,params, *args):
  clf = model()
  clf.set_params(**params)
  clf.fit(args[0], args[1])
  preds = clf.predict(args[2])
  recall = recall_score(args[3], preds)
  precision = precision_score(args[3], preds)
  f1 = f1_score(args[3], preds)
  
  result = {
    'model_name': [model.__name__] ,
    'params': [params],
    'recall': [recall],
    'precision': [precision],
    'f1': [f1]
  }
  
  df_result = pd.DataFrame(result)
  df_result.to_csv(f'model_results/{args[4]}.csv', index = False)
  
  return df_result



def get_best_features(X, y, models, cv):

  for model in models:
    
    pipeline = Pipeline([('ss', StandardScaler()),
                    ('', model())])

    gs = GridSearchCV(
                      pipeline,
                      param_grid=[models[model]],
                      scoring='f1',
                      cv=cv
    )
    
    gs.fit(X, y)
    
    print(f'model: {model.__name__}')
    print(f'\t best params: {gs.best_params_}')
    print(f'\t best f1 score: {gs.best_score_}')
  
    return 



In [8]:
# Carga de Data
data = pd.read_csv('Clothstore_data.csv')
target = pd.read_csv('Clothstore_target.csv')
features  = one_hot_encoder(data)

In [79]:
# Determinando mejores variables 

params = {
  RandomForestClassifier:{
    '__n_estimators': [10,15,30,50,100,200],
    '__max_depth':[5,10,20,50,100,200],
    '__min_samples_split': [2, 5, 10]
    },
  LogisticRegression:{
    '__penalty': ['none','l1','l2','elasticnet'],
    '__C':[100, 10, 1.0, 0.1, 0.01]
    }
}


get_best_features(features,target, params, 3)

model: RandomForestClassifier
	 best params: {'__max_depth': 20, '__min_samples_split': 5, '__n_estimators': 50}
	 best f1 score: 0.3857280570333803


In [32]:
best_params = {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 50}

rf = RandomForestClassifier()
rf.set_params(**best_params)
rf.fit(features, target)
temp_df = list(zip(features.columns, rf.feature_importances_))
temp_df = pd.DataFrame(temp_df, columns =['Feature', 'Importance'])
top_feature_importance = temp_df.sort_values(by= 'Importance', ascending= False).iloc[0:10,[0]]
top_feature_importance = list(top_feature_importance["Feature"])
top_feature_importance

['num_days_between_purchases_lifetime',
 'num_individual_items_purchased',
 'num_days_between_purchases',
 'num_visits_customer',
 'num_days_customer',
 'num_diff_product_classes',
 'total_net_sales',
 'avg_net_sales_per_visit',
 'product_uniformity',
 'ammount_spend_store_3']

In [34]:
params = {
  RandomForestClassifier:{
    '__n_estimators': [10,15,30,50,100,200],
    '__max_depth':[5,10,20,50,100,200],
    '__min_samples_split': [2, 5, 10]
    },
  LogisticRegression:{
    '__penalty': ['none','l1','l2','elasticnet'],
    '__C':[100, 10, 1.0, 0.1, 0.01]
    }
}


get_best_features(features[top_feature_importance],target, params, 3)


model: RandomForestClassifier
	 best params: {'__max_depth': 20, '__min_samples_split': 10, '__n_estimators': 15}
	 best f1 score: 0.4170738601706471


In [14]:
# cross validation

df = one_hot_encoder(data)

kf = KFold(n_splits= 5)
metrics = []


for model in models:
  print(model)
  for i, param in enumerate(models[model]):
    print(param)
    num_iterator = 1
    for train, test in kf.split(df):
      
      scaler = StandardScaler()
      scaler.fit(df.iloc[train,:])
      
      result = fitter(
                  model,
                  param,
                  scaler.transform(df.iloc[train,:]),
                  target.iloc[train,:],
                  scaler.transform(df.iloc[test,:]),
                  target.iloc[test,:],
                  f'{model.__name__}_{i}_{num_iterator}'
                                    
      )
      
      num_iterator =  num_iterator + 1
    
      

<class 'sklearn.neighbors._classification.KNeighborsClassifier'>
{'n_neighbors': 2}
{'n_neighbors': 3}
{'n_neighbors': 5}
{'n_neighbors': 10}
{'n_neighbors': 15}
{'n_neighbors': 20}
<class 'sklearn.tree._classes.DecisionTreeClassifier'>
{'max_depth': 5}
{'max_depth': 7}
{'max_depth': 15}
{'max_depth': 30}
{'max_depth': 50}
{'max_depth': 100}
<class 'sklearn.ensemble._forest.RandomForestClassifier'>
{'n_estimators': 10, 'max_depth': 10}
{'n_estimators': 20, 'max_depth': 10}
{'n_estimators': 50, 'max_depth': 10}
{'n_estimators': 100, 'max_depth': 10}
{'n_estimators': 10, 'max_depth': 30}
{'n_estimators': 20, 'max_depth': 30}
{'n_estimators': 50, 'max_depth': 30}
{'n_estimators': 100, 'max_depth': 30}
{'n_estimators': 10, 'max_depth': 50}
{'n_estimators': 20, 'max_depth': 50}
{'n_estimators': 50, 'max_depth': 50}
{'n_estimators': 100, 'max_depth': 50}
{'n_estimators': 10, 'max_depth': 100}
{'n_estimators': 20, 'max_depth': 100}
{'n_estimators': 50, 'max_depth': 100}
{'n_estimators': 100, 

In [15]:
# Reporte con los resultados
files =  [f for f in os.listdir("./model_results") if f.endswith(".csv")]  

frames = []
for file in files:
  frame = pd.read_csv(f'model_results/{file}')
  frames.append(frame)
  
results = pd.concat(frames, axis= 0, ignore_index= True)

In [16]:
import plotly.express as px

for model in results['model_name'].unique():
  fig = px.box(results[results['model_name'] == model],
               x="params", 
               y="f1",
               points="all",
               title=f"Modelo: {model}")
  fig.show()

# RESULTADO FINAL MEJOR MODELO
En la siguiente celda, escriba el mejor modelo encontrado por usted, de forma que el profesor y su ayudante puedan evaluar el performance de este modelo escogido ante nuevos datos.  Ponga por lo tanto todos los pasos requeridos para poder hacer esto, incluyendo el preprocesing que sea necesario.

**Ejemplo**: si el modelo final fuera aplicar KNN con 8-vecinos usando los datos normalizados y sin la columna `has_valid_phone`, entonces debería ser algo así:


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('Clothstore_data.csv')
target = pd.read_csv('Clothstore_target.csv')

#Preprocessing data
del data['has_valid_phone']
scaler = StandardScaler()
dataFinal = scaler.fit_transform(data)

#Mejor modelo final
modelo_final = KNeighborsClassifier(n_neighbors=8)
modelo_final.fit(dataFinal, target)